# Estimativa preços de venda do dataset Boston_housing

## Importação dos pacotes

In [1]:
import numpy as np

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torch.autograd import Variable
import torch.nn.functional as F

from lib.pytorch_trainer import DeepNetTrainer

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

GPU available: True


## Dataset - o arquivo foi gerado pelo notebook:

Leitura e normalização do Dataset: [boston_housing.ipynb](boston_housing.ipynb)

In [2]:
datain = np.load('../data/boston_housing_normalize.npz')

x, y = datain['Xtra'], datain['ytra']

## Dividindo em treino (80%) e validação (20%)

In [3]:
i = np.arange(len(x))
np.random.shuffle(i)
n_train = int(0.8 * len(x))
i_train = i[:n_train]
i_test  = i[n_train:]

In [4]:
x_train = x[:n_train]
y_train = y[:n_train]
x_test  = x[n_train:]
y_test  = y[n_train:]

In [5]:
n_samples, n_attributes = x_train.shape
print('n_samples:', n_samples)
print('n_attributes:', n_attributes)

n_samples: 404
n_attributes: 13


## Criando datasets e loaders

In [6]:
data_train = TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))
data_eval  = TensorDataset(torch.FloatTensor(x_test) , torch.FloatTensor(y_test))
loader_train = DataLoader(data_train,batch_size=len(y_train)//2,shuffle=True)
loader_eval  = DataLoader(data_eval, batch_size=len(y_test))

## Rede, uma camada de escondida de 40 neurônios

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(n_attributes, 40)
        self.layer2 = nn.Linear(40, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.layer2(x)
        return x

model = Model()
if use_gpu:
    model.cuda()
model

Model (
  (layer1): Linear (13 -> 40)
  (layer2): Linear (40 -> 1)
)

## Parâmetros do otimizador

In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9, nesterov=True)
criterion = nn.MSELoss()
trainer = DeepNetTrainer(
        #file_basename = '../../models/bostonhousing_pt',
        model =         model,
        criterion =     criterion,
        optimizer =     optimizer,
        #callbacks =     [MyCallback()],
        #reset =         False
        )

## Treinamento

In [12]:
trainer.fit_loader(300, loader_train, loader_eval)